In [46]:
import pandas as pd
import time

In [61]:
streaming_services = {'hulu':"data/hulu_titles.csv", \
                      'amazon_prime':"data/amazon_prime_titles.csv", \
                      'netflix':"data/netflix_titles.csv", \
                      'disney_plus':"data/disney_plus_titles.csv"}

df_list = list()

for ss in streaming_services.keys():
    ss_df = pd.read_csv(streaming_services[ss])
    ss_df['streaming_service'] = ss
    df_list.append(ss_df)

df = pd.concat(df_list)

df.to_csv("data/streaming_titles.csv",index=False)


In [48]:
import requests
print('start')
rapid_api_key = 'API KEY HERE'
def imdbpuller(rapid_api_key,title,year):
    url = "https://mdblist.p.rapidapi.com/"

    querystring = {"s":title,"y":year}

    headers = {
        "X-RapidAPI-Key": rapid_api_key,
        "X-RapidAPI-Host": "mdblist.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)

    return response


start


In [50]:
print(imdbpuller(rapid_api_key,"Breaking Bad",2008).json()["search"][0])

{'id': 'tt0903747', 'title': 'Breaking Bad', 'year': 2008, 'score': 92, 'score_average': 92, 'type': 'show', 'imdbid': 'tt0903747', 'tmdbid': 1396, 'traktid': 1388, 'tvdbid': 81189}


In [58]:
imdbscores =  dict()

for row in df.head(50).iloc():
    title = row["title"]
    year = row["release_year"]
    responded = False
    count = 0

    while not responded:
        if count >= 20:
            print("response failed for {title}".format(title=title))
            break
        currjson = imdbpuller(rapid_api_key,title,year).json()
        responded = currjson['response']
        if not responded:
            time.sleep(0.25)
        count+=1
    if not responded:
        continue

    madeit = False
    for queries in currjson['search']:
        if queries['title'] == title:
            imdbscores[title] = queries['score']
            print('success for {title}'.format(title=title))
            madeit = True
            break
    if not madeit:
        print("Could not find valid match for {title}".format(title=title))

print(imdbscores)

success for Ricky Velez: Here's Everything
success for Silent Night
success for The Marksman
success for Gaia
success for Settlers
Could not find valid match for The Halloween Candy Magic Pet
Could not find valid match for The Evil Next Door
success for The Next Thing You Eat
success for Queens
success for The Bachelorette
Could not find valid match for The Real Queens of Hip-Hop: The Women Who Changed the Game
success for Dream Horse
success for Out of the Shadows: The Man Behind the Steele Dossier
success for Wakefield
success for Home Sweet Home
Could not find valid match for Showtime Championship Boxing: Lopez vs. Salido (R)
Could not find valid match for Showtime Championship Boxing: Wilder vs. Molina
success for America's Book of Secrets
success for Beyond Oak Island
success for Beyond Scared Straight
success for Cheer Camp Killer
success for Hoarders
success for Little Women: Atlanta
success for Marrying Millions
success for Nightwatch
Could not find valid match for Seven Year S

KeyboardInterrupt: 

In [60]:
print(len(imdbscores))

160
